In [1]:
import polars as pl
import numpy as np
import warnings
import matplotlib.pyplot as plt
from datetime import timedelta
import sys
import polars as pl
import numpy as np
import pandas as pd
import scipy

print(f"Python version: {sys.version}")
print(f"Polars version: {pl.__version__}")
print(f"NumPy version: {np.__version__}")
print(f"Pandas version: {pd.__version__}")
print(f"SciPy version: {scipy.__version__}")

print(f"Polars version: {pl.__version__}")
warnings.filterwarnings("ignore")
commission = 0.25 / 10000.0
USE_HOUR_DATA = 0

Python version: 3.11.4 (main, Jul  5 2023, 13:45:01) [GCC 11.2.0]
Polars version: 0.20.17
NumPy version: 1.26.4
Pandas version: 1.5.3
SciPy version: 1.10.1
Polars version: 0.20.17


In [2]:
if USE_HOUR_DATA:
    cur_path = "data/agg_data_hour_to_day.parquet" 
    cur_path = "data/agg_data_hour_to_day_alpha101.parquet" 
    input_df = pl.read_parquet(cur_path)


    # Check if 'amihud' column exists in input_df
    if 'amihud' in input_df.columns:
        input_df = input_df.drop('amihud')
else:
    path = "data/kline_1d_df.parquet"
    path = 'data/pure_day_data_agg_from_hour_sep29.parquet'
    path = 'data/all_data_1d_boris_converted.parquet'
    input_df = pl.read_parquet(path)
    if 'jj_code' in input_df.columns:
        input_df = input_df.rename({"jj_code": "symbol"})
    # input_df = pl.read_parquet("data/daily_data.pa")

INPUT_PATH = path
print (f'input path: {INPUT_PATH}')

if 1:
    # add hour_group
    input_df = input_df.with_columns (
        pl.col ('open_time').dt.hour().alias ('hour_group')
    )

    # Move the 'hour_group' column to the front
    input_df = input_df.select(
        pl.col('hour_group'),
        pl.all().exclude('hour_group')
    )



input_df = input_df.sort(['symbol', 'open_time'])

print (f'input df: {input_df}')

# # MA 10
# calc_factor = input_df.groupby('symbol').apply(
#     lambda df: df.with_columns(
#         (pl.col('close').rolling_mean(window_size=10)).alias('factor')
#         # (pl.col('close')).alias('factor')
#     )
# )

# # the return in the past 3 bar
# calc_factor = input_df.groupby('symbol').apply(
#     lambda df: df.with_columns(
#         # Calculate the return percentage compared to 10 time slices ago
#         ((pl.col('close') / pl.col('close').shift(3) - 1) * 100).alias('factor')
#     )
# )

# input_df = input_df.with_columns(
#     ((pl.col('close') / pl.col('close').shift(1) - 1) * 100).alias('return')
# )

input_df = input_df.sort(["symbol", "open_time"])
input_column_name = input_df.columns

print (input_df.columns)


# 千万注意，这里是计算此时刻相对于前一时刻的return，不能使用未来信息 -> shift(1)是整体往下移动
if USE_HOUR_DATA:
    rows_per_day = 24
else:
    rows_per_day = 1

input_df = input_df.with_columns(
    ((pl.col("close") / pl.col("close").shift(rows_per_day) - 1) * 100)
    .over("symbol")  # Applying the function over each symbol group
    .alias("past_day_return")
)

input_df = input_df.with_columns(
    ((pl.col("close") / pl.col("close").shift(1) - 1) * 100)
    .over("symbol")  # Applying the function over each symbol group
    .alias("past_row_return")
)



# 暂时把过去一天的return作为return列
input_df = input_df.with_columns(
    pl.col("past_day_return").alias("return")
)



# below calc amihud
if 'amihud' not in set (input_df.columns):
    print ('calc amihud')

    input_df = input_df.with_columns(
        pl.col("return").abs().rolling_sum(window_size=10).over("symbol", "hour_group").alias("rolling_abs_return_sum"),
        pl.col("quote_volume").rolling_sum(window_size=10).over("symbol", "hour_group").alias("rolling_quote_volume_sum")
    )

    input_df = input_df.with_columns(
        (pl.col("rolling_abs_return_sum") / pl.col("rolling_quote_volume_sum"))
        .over("symbol", "hour_group")  # Apply the final operation within each symbol group
        .alias("amihud")
    )
    input_df = input_df.drop(["rolling_abs_return_sum"])
    input_df = input_df.drop([ "rolling_quote_volume_sum"])
    # input_df = input_df.drop([ "rolling_volume_sum"])


input_df

input path: data/all_data_1d_boris_converted.parquet
input df: shape: (228_254, 16)
┌────────────┬────────────┬───────────┬───────────┬───┬───────────┬───────────┬────────┬───────────┐
│ hour_group ┆ open_time  ┆ symbol    ┆ open_time ┆ … ┆ taker_buy ┆ taker_buy ┆ ignore ┆ close_tim │
│ ---        ┆ ---        ┆ ---       ┆ _ms       ┆   ┆ _volume   ┆ _quote_vo ┆ ---    ┆ e         │
│ i8         ┆ datetime[m ┆ str       ┆ ---       ┆   ┆ ---       ┆ lume      ┆ str    ┆ ---       │
│            ┆ s]         ┆           ┆ i64       ┆   ┆ f64       ┆ ---       ┆        ┆ datetime[ │
│            ┆            ┆           ┆           ┆   ┆           ┆ f64       ┆        ┆ ms]       │
╞════════════╪════════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪════════╪═══════════╡
│ 0          ┆ 2023-11-22 ┆ 1000BONKU ┆ 170061120 ┆ … ┆ 6.1849e9  ┆ 2.7467e7  ┆ 0      ┆ 2023-11-2 │
│            ┆ 00:00:00   ┆ SDT       ┆ 0000      ┆   ┆           ┆           ┆        ┆ 2 23:59:5 │
│      

hour_group,open_time,symbol,open_time_ms,open,high,low,close,volume,close_time_ms,quote_volume,count,taker_buy_volume,taker_buy_quote_volume,ignore,close_time,past_day_return,past_row_return,return,amihud
i8,datetime[ms],str,i64,f64,f64,f64,f64,f64,i64,f64,i64,f64,f64,str,datetime[ms],f64,f64,f64,f64
0,2023-11-22 00:00:00,"""1000BONKUSDT""",1700611200000,0.00478,0.004825,0.004076,0.004531,1.2701e10,1700697599999,5.6368e7,320715,6.1849e9,2.7467e7,"""0""",2023-11-22 23:59:59.999,null,null,null,null
0,2023-11-23 00:00:00,"""1000BONKUSDT""",1700697600000,0.004531,0.004858,0.00393,0.004267,1.8971e10,1700783999999,8.1590e7,573386,8.8982e9,3.8318e7,"""0""",2023-11-23 23:59:59.999,-5.826528,-5.826528,-5.826528,null
0,2023-11-24 00:00:00,"""1000BONKUSDT""",1700784000000,0.004267,0.004335,0.003835,0.00414,1.7169e10,1700870399999,6.9929e7,475254,7.9410e9,3.2393e7,"""0""",2023-11-24 23:59:59.999,-2.97633,-2.97633,-2.97633,null
0,2023-11-25 00:00:00,"""1000BONKUSDT""",1700870400000,0.004143,0.0046,0.004084,0.00434,1.3093e10,1700956799999,5.6988e7,501832,6.3891e9,2.7819e7,"""0""",2023-11-25 23:59:59.999,4.830918,4.830918,4.830918,null
0,2023-11-26 00:00:00,"""1000BONKUSDT""",1700956800000,0.004341,0.004473,0.0039,0.004039,8.2273e9,1701043199999,3.4347e7,344203,3.8247e9,1.5980e7,"""0""",2023-11-26 23:59:59.999,-6.935484,-6.935484,-6.935484,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
0,2024-09-26 00:00:00,"""ZRXUSDT""",1727308800000,0.3472,0.3578,0.3392,0.3561,3.3656e7,1727395199999,1.1738e7,71748,1.6593e7,5.7885e6,"""0""",2024-09-26 23:59:59.999,2.592913,2.592913,2.592913,3.1322e-7
0,2024-09-27 00:00:00,"""ZRXUSDT""",1727395200000,0.3561,0.371,0.3469,0.3671,3.4971e7,1727481599999,1.2599e7,78393,1.7560e7,6.3213e6,"""0""",2024-09-27 23:59:59.999,3.08902,3.08902,3.08902,2.9494e-7
0,2024-09-28 00:00:00,"""ZRXUSDT""",1727481600000,0.3672,0.3691,0.3488,0.3555,1.6642e7,1727567999999,5.9484e6,46137,8003898.2,2.8619e6,"""0""",2024-09-28 23:59:59.999,-3.159902,-3.159902,-3.159902,2.8556e-7


In [3]:
input_df['symbol'].unique().to_list()

['LEVERUSDT',
 'CFXUSDT',
 'SUIUSDT',
 'DUSKUSDT',
 'BNXUSDT',
 'USTCUSDT',
 'DGBUSDT',
 'ENSUSDT',
 'PIXELUSDT',
 'CTKUSDT',
 'IOTAUSDT',
 'ETHWUSDT',
 'EGLDUSDT',
 'BELUSDT',
 'AUCTIONUSDT',
 'ZECUSDT',
 'PENDLEUSDT',
 'IMXUSDT',
 'QTUMUSDT',
 'STGUSDT',
 'OPUSDT',
 'JUPUSDT',
 'FRONTUSDT',
 'FILUSDT',
 'DENTUSDT',
 'REEFUSDT',
 'LRCUSDT',
 'ZILUSDT',
 'DEFIUSDT',
 'KNCUSDT',
 'SANDUSDT',
 'CKBUSDT',
 'BIGTIMEUSDT',
 'SRMUSDT',
 'BOMEUSDT',
 'CELOUSDT',
 'KEYUSDT',
 'DYMUSDT',
 'EOSUSDT',
 'WOOUSDT',
 'RDNTUSDT',
 'VANRYUSDT',
 'OMGUSDT',
 'CTSIUSDT',
 'SNXUSDT',
 'STMXUSDT',
 'ATAUSDT',
 'ETCUSDT',
 'SLPUSDT',
 'RONINUSDT',
 'JTOUSDT',
 'AVAXUSDT',
 'ACEUSDT',
 'SCUSDT',
 'CELRUSDT',
 'NEARUSDT',
 'C98USDT',
 'BANDUSDT',
 'ICPUSDT',
 'NEOUSDT',
 'KASUSDT',
 'ETHUSDT',
 'YGGUSDT',
 'XTZUSDT',
 'ZKUSDT',
 'ORBSUSDT',
 'INJUSDT',
 'DOTUSDT',
 'ENAUSDT',
 'BEAMXUSDT',
 'DARUSDT',
 'GLMRUSDT',
 'PEOPLEUSDT',
 'OXTUSDT',
 'OMUSDT',
 'BTSUSDT',
 'LISTAUSDT',
 'COTIUSDT',
 'ONDOUSDT',
 'TON

In [4]:
print (input_df.columns, type(input_df.columns))
input_df.filter ((pl.col ('symbol') == 'NOTUSDT') & (pl.col ('hour_group') == 0)).select (pl.col (input_df.columns[:4] + ['return', 'past_day_return', 'past_row_return'])).head (20)

['hour_group', 'open_time', 'symbol', 'open_time_ms', 'open', 'high', 'low', 'close', 'volume', 'close_time_ms', 'quote_volume', 'count', 'taker_buy_volume', 'taker_buy_quote_volume', 'ignore', 'close_time', 'past_day_return', 'past_row_return', 'return', 'amihud'] <class 'list'>


hour_group,open_time,symbol,open_time_ms,return,past_day_return,past_row_return
i8,datetime[ms],str,i64,f64,f64,f64
0,2024-05-16 00:00:00,"""NOTUSDT""",1715817600000,null,null,null
0,2024-05-17 00:00:00,"""NOTUSDT""",1715904000000,-7.813535,-7.813535,-7.813535
0,2024-05-18 00:00:00,"""NOTUSDT""",1715990400000,-4.927453,-4.927453,-4.927453
0,2024-05-19 00:00:00,"""NOTUSDT""",1716076800000,-16.183137,-16.183137,-16.183137
0,2024-05-20 00:00:00,"""NOTUSDT""",1716163200000,2.614026,2.614026,2.614026
…,…,…,…,…,…,…
0,2024-05-31 00:00:00,"""NOTUSDT""",1717113600000,-7.497226,-7.497226,-7.497226
0,2024-06-01 00:00:00,"""NOTUSDT""",1717200000000,35.966415,35.966415,35.966415
0,2024-06-02 00:00:00,"""NOTUSDT""",1717286400000,42.822936,42.822936,42.822936


In [5]:
OUTPUT_PATH = INPUT_PATH.replace('.parquet', '_with_amihud.parquet')
input_df.write_parquet(OUTPUT_PATH)
print (f'write output file to {OUTPUT_PATH}')
input_df

write output file to data/all_data_1d_boris_converted_with_amihud.parquet


hour_group,open_time,symbol,open_time_ms,open,high,low,close,volume,close_time_ms,quote_volume,count,taker_buy_volume,taker_buy_quote_volume,ignore,close_time,past_day_return,past_row_return,return,amihud
i8,datetime[ms],str,i64,f64,f64,f64,f64,f64,i64,f64,i64,f64,f64,str,datetime[ms],f64,f64,f64,f64
0,2023-11-22 00:00:00,"""1000BONKUSDT""",1700611200000,0.00478,0.004825,0.004076,0.004531,1.2701e10,1700697599999,5.6368e7,320715,6.1849e9,2.7467e7,"""0""",2023-11-22 23:59:59.999,null,null,null,null
0,2023-11-23 00:00:00,"""1000BONKUSDT""",1700697600000,0.004531,0.004858,0.00393,0.004267,1.8971e10,1700783999999,8.1590e7,573386,8.8982e9,3.8318e7,"""0""",2023-11-23 23:59:59.999,-5.826528,-5.826528,-5.826528,null
0,2023-11-24 00:00:00,"""1000BONKUSDT""",1700784000000,0.004267,0.004335,0.003835,0.00414,1.7169e10,1700870399999,6.9929e7,475254,7.9410e9,3.2393e7,"""0""",2023-11-24 23:59:59.999,-2.97633,-2.97633,-2.97633,null
0,2023-11-25 00:00:00,"""1000BONKUSDT""",1700870400000,0.004143,0.0046,0.004084,0.00434,1.3093e10,1700956799999,5.6988e7,501832,6.3891e9,2.7819e7,"""0""",2023-11-25 23:59:59.999,4.830918,4.830918,4.830918,null
0,2023-11-26 00:00:00,"""1000BONKUSDT""",1700956800000,0.004341,0.004473,0.0039,0.004039,8.2273e9,1701043199999,3.4347e7,344203,3.8247e9,1.5980e7,"""0""",2023-11-26 23:59:59.999,-6.935484,-6.935484,-6.935484,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
0,2024-09-26 00:00:00,"""ZRXUSDT""",1727308800000,0.3472,0.3578,0.3392,0.3561,3.3656e7,1727395199999,1.1738e7,71748,1.6593e7,5.7885e6,"""0""",2024-09-26 23:59:59.999,2.592913,2.592913,2.592913,3.1322e-7
0,2024-09-27 00:00:00,"""ZRXUSDT""",1727395200000,0.3561,0.371,0.3469,0.3671,3.4971e7,1727481599999,1.2599e7,78393,1.7560e7,6.3213e6,"""0""",2024-09-27 23:59:59.999,3.08902,3.08902,3.08902,2.9494e-7
0,2024-09-28 00:00:00,"""ZRXUSDT""",1727481600000,0.3672,0.3691,0.3488,0.3555,1.6642e7,1727567999999,5.9484e6,46137,8003898.2,2.8619e6,"""0""",2024-09-28 23:59:59.999,-3.159902,-3.159902,-3.159902,2.8556e-7
